In [24]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch import nn, optim
import os
from PIL import Image
import numpy as np

In [22]:
# Directory for data
dataset_dir = '/Users/Sara/Desktop/FashionExpert-SE/FashionExpert/Dataset'


# transformations for the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
dataset = ImageFolder(root=dataset_dir, transform=transform)

# training and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)
print(numpy.__version__)

1.26.4


In [25]:
# pre-trained MobileNetV3 Large 
model = models.mobilenet_v3_large(pretrained=True)

# last layer => how many classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(dataset.classes))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        total = 0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Epoch {epoch+1}, Validation Accuracy: {100 * correct / total}%')

train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)

torch.save(model.state_dict(), 'trained_model.pth')

RuntimeError: Numpy is not available

In [ ]:
# Function to load an image and predict its class
def predict_image(image_path, model, transform):
    image = Image.open(image_path)
    image = transform(image).float()
    image = image.unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        idx = predicted.item()
    return dataset.classes[idx]

# Load the trained model (if not already in memory)
model.load_state_dict(torch.load('trained_model.pth'))
model.to(device)

# Example usage
image_path = 'C:\Users\Sara\Desktop\FashionExpert-SE\FashionExpert\image1.jpg'
predicted_class = predict_image(image_path, model, transform)
print(f'Predicted class: {predicted_class}')